In [1]:
# Vammos a calcular las anchor boxes para YOLOv3 usando K-means con IoU como métrica de distancia.
# Este script asume que tienes un archivo CSV con las anotaciones de las bounding boxes.
# Asegúrate de que tienes las librerías necesarias instaladas:

import os
import pandas as pd
import numpy as np
import torch
from tqdm import tqdm # Para barras de progreso
import random # Para inicialización de K-means

# Asegúrate de que la función intersection_over_union esté accesible
# Si la tienes en utils.py, impórtala:
# from utils import intersection_over_union
# Si no, la definimos aquí para que el script sea autocontenido:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2
    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
    else:
        raise ValueError("box_format debe ser 'midpoint' o 'corners'")

    # Calcular las coordenadas del rectángulo de intersección
    # Aseguramos que estas variables estén definidas después de los ifs
    x1_inter = torch.max(box1_x1, box2_x1)
    y1_inter = torch.max(box1_y1, box2_y1)
    x2_inter = torch.min(box1_x2, box2_x2)
    y2_inter = torch.min(box1_y2, box2_y2)

    # Calcular el área de intersección
    intersection = (x2_inter - x1_inter).clamp(0) * \
                    (y2_inter - y1_inter).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    union = box1_area + box2_area - intersection + 1e-6
    iou = intersection / union
    return iou


def iou_distance(box, centroid):
    """
    Calcula la "distancia" como 1 - IoU, para usar en K-means.
    Las cajas se asumen centradas en (0,0) para comparar solo dimensiones.
    Args:
        box (np.array): [width, height] de una bounding box.
        centroid (np.array): [width, height] de un centroide (anchor).
    Returns:
        float: 1 - IoU entre la caja y el centroide.
    """
    box_tensor = torch.tensor([[0.0, 0.0, box[0], box[1]]], dtype=torch.float32)
    centroid_tensor = torch.tensor([[0.0, 0.0, centroid[0], centroid[1]]], dtype=torch.float32)
    
    # Usamos box_format="corners" porque las estamos tratando como (x1, y1, x2, y2)
    # donde x1,y1 son 0 y x2,y2 son width, height.
    iou = intersection_over_union(box_tensor, centroid_tensor, box_format="corners").item()
    return 1 - iou # Queremos minimizar la "distancia", que es 1 - IoU

def calculate_average_iou(boxes, centroids):
    """
    Calcula el IoU promedio entre las cajas y sus centroides asignados.
    """
    total_iou = 0.0
    num_boxes = 0
    for box in boxes:
        best_iou = -1
        for centroid in centroids:
            iou = 1 - iou_distance(box, centroid)
            if iou > best_iou:
                best_iou = iou
        total_iou += best_iou
        num_boxes += 1
    return total_iou / num_boxes if num_boxes > 0 else 0.0


def kmeans_iou(boxes, k, max_iters=100, tol=1e-6, random_seed=42):
    """
    Implementación de K-means que utiliza 1 - IoU como métrica de distancia.
    Args:
        boxes (np.array): Array de dimensiones de bounding boxes (N, 2), donde N es el número de cajas.
                        Cada fila es [width, height].
        k (int): Número de clusters (número de anchor boxes a generar).
        max_iters (int): Número máximo de iteraciones.
        tol (float): Tolerancia para la convergencia.
        random_seed (int): Semilla para la reproducibilidad.
    Returns:
        np.array: Centroides finales (anchor boxes).
    """
    # 1. Inicialización de los centroides (K-means++ como aproximación simplificada: elige k puntos aleatorios)
    # Opcional: Para una inicialización más robusta, podrías implementar K-means++ completo
    random.seed(random_seed)
    centroids = np.array(random.sample(list(boxes), k)).astype(np.float32) # Convertir a float32

    print(f"Iniciando K-means con {k} centroides iniciales:")
    print(centroids)

    for iteration in tqdm(range(max_iters), desc="Ejecutando K-means"):
        # 2. Asignación: Asigna cada caja al centroide más cercano (con menor 1-IoU)
        clusters = [[] for _ in range(k)]
        for i, box in enumerate(boxes):
            distances = [iou_distance(box, centroid) for centroid in centroids]
            closest_centroid_idx = np.argmin(distances)
            clusters[closest_centroid_idx].append(box)

        # 3. Actualización: Recalcula los centroides (promedio de las cajas asignadas)
        new_centroids = np.zeros_like(centroids)
        for i, cluster in enumerate(clusters):
            if cluster: # Evitar división por cero si un cluster está vacío
                new_centroids[i] = np.mean(cluster, axis=0)
            else:
                # Si un cluster está vacío, re-inicializarlo con una caja aleatoria
                new_centroids[i] = random.choice(list(boxes))

        # 4. Comprobar convergencia
        # Calculamos el desplazamiento máximo de los centroides
        # Usamos 1 - IoU como "distancia" entre centroides para la convergencia
        max_centroid_shift = 0.0
        for i in range(k):
            shift = iou_distance(centroids[i], new_centroids[i])
            if shift > max_centroid_shift:
                max_centroid_shift = shift
        
        centroids = new_centroids

        if max_centroid_shift < tol:
            print(f"K-means convergió en la iteración {iteration + 1}.")
            break
    else:
        print(f"K-means alcanzó el número máximo de iteraciones ({max_iters}) sin converger con la tolerancia {tol}.")

    return centroids

if __name__ == "__main__":
    # --- Configuración ---
    DATA_ROOT = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset'
    CSV_FILE = os.path.join(DATA_ROOT, 'annotations.csv')
    NUM_ANCHORS = 9 # Para YOLOv3, suelen ser 9 anchors
    
    # --- 1. Cargar las dimensiones de todas las bounding boxes ---
    print(f"Cargando anotaciones desde: {CSV_FILE}")
    try:
        annotations_df = pd.read_csv(CSV_FILE)
        # Asegurarse de que no haya NaN y convertir a int
        annotations_df = annotations_df.dropna(subset=['xmin', 'ymin', 'xmax', 'ymax'])
        annotations_df[['xmin', 'ymin', 'xmax', 'ymax']] = annotations_df[['xmin', 'ymin', 'xmax', 'ymax']].astype(int)
    except FileNotFoundError:
        print(f"Error: Archivo CSV no encontrado en {CSV_FILE}")
        exit()

    # Calcular ancho y alto de cada caja
    # original_w y original_h son necesarios para desnormalizar si el CSV ya está normalizado.
    # En este dataset BCCD, las coordenadas son en píxeles.
    annotations_df['width'] = annotations_df['xmax'] - annotations_df['xmin']
    annotations_df['height'] = annotations_df['ymax'] - annotations_df['ymin']

    # Filtrar cajas degeneradas (ancho o alto <= 0)
    filtered_boxes = annotations_df[(annotations_df['width'] > 0) & (annotations_df['height'] > 0)]

    # Extraer solo las dimensiones [width, height]
    # Asegúrate de que las dimensiones sean coherentes con tu img_size si están normalizadas.
    # Si tu CSV tiene (x,y,w,h) normalizados, deberías multiplicarlos por el tamaño de la imagen original
    # para obtener dimensiones en píxeles para el clustering.
    # Como BCCD tiene píxeles, ya está bien.
    all_box_dimensions = filtered_boxes[['width', 'height']].values.astype(np.float32)

    if len(all_box_dimensions) == 0:
        print("No se encontraron bounding boxes válidas para el clustering. Revisa tu CSV y filtros.")
        exit()

    print(f"Total de {len(all_box_dimensions)} bounding boxes válidas cargadas para clustering.")

    # --- 2. Ejecutar K-means con métrica IoU ---
    print(f"\nEjecutando K-means para encontrar {NUM_ANCHORS} anchor boxes...")
    anchor_centroids = kmeans_iou(all_box_dimensions, k=NUM_ANCHORS, max_iters=300, tol=1e-5)

    # --- 3. Ordenar y Formatear los resultados ---
    # Ordenar los anchors por tamaño (ej. área o ancho) para asignarlos a las escalas.
    # YOLOv3 típicamente usa los 3 más grandes para la escala más pequeña (grid 13x13),
    # los 3 medianos para la escala media (grid 26x26), y los 3 más pequeños para la escala más grande (grid 52x52).
    
    # Ordenar por área para asegurar que los anchors más grandes vayan a la escala adecuada.
    # anchor_centroids es np.array([[w1,h1], [w2,h2], ...])
    anchor_centroids = anchor_centroids[np.argsort(anchor_centroids[:, 0] * anchor_centroids[:, 1])]
    
    # Dividir en 3 grupos (para 3 escalas) y formatear
    # Asignación sugerida:
    # Scale 0 (grid más pequeño, stride 32): 3 anchors más grandes
    # Scale 1 (grid medio, stride 16): 3 anchors medianos
    # Scale 2 (grid más grande, stride 8): 3 anchors más pequeños

    # Invertir el orden para que los más grandes estén primero
    anchor_centroids = anchor_centroids[::-1] 

    # Agrupar para las 3 escalas de YOLOv3 (si NUM_ANCHORS es 9)
    # anchors_per_scale = NUM_ANCHORS // 3
    # anchors_scale0 = anchor_centroids[0:anchors_per_scale]
    # anchors_scale1 = anchor_centroids[anchors_per_scale:2*anchors_per_scale]
    # anchors_scale2 = anchor_centroids[2*anchors_per_scale:3*anchors_per_scale]

    # Para YOLOv3, se suelen usar 3 anclas por escala. Las anclas se pasan a la Loss Function
    # como una lista de listas de tuplas: [[(w,h), (w,h), (w,h)], ... ]
    # y en el orden de las escalas de la red:
    # [anchors_for_scale_0 (largest grid), anchors_for_scale_1 (medium grid), anchors_for_scale_2 (smallest grid)]
    # NO: el orden de los anclas es Smallest grid -> largest objects. Esto significa que la primera sub-lista
    # de ANCHORS debería corresponder a la capa de salida con el stride más grande (e.g., 13x13 grid for 416x416).
    # Esa capa detecta objetos grandes.

    # Ordenarlos por área y agruparlos de mayor a menor para asignarlos a las escalas
    # Los más grandes (índices 0, 1, 2) van a la escala 0 (grid 13x13)
    # Los medianos (índices 3, 4, 5) van a la escala 1 (grid 26x26)
    # Los más pequeños (índices 6, 7, 8) van a la escala 2 (grid 52x52)

    # Aseguramos el orden de las anclas de mayor a menor
    sorted_anchors = []
    for anchor in anchor_centroids:
        sorted_anchors.append(tuple(anchor.round(2).astype(int))) # Redondeamos y convertimos a entero para tuplas

    # Formato final para la Loss Function
    # Este es el formato de ANCHORS que se pasa a YOLOv3Loss
    final_anchors = [
        sorted_anchors[0:3], # Las 3 más grandes (para grid 13x13)
        sorted_anchors[3:6], # Las 3 medianas (para grid 26x26)
        sorted_anchors[6:9], # Las 3 más pequeñas (para grid 52x52)
    ]
    
    print("\n--- Anchor Boxes Calculadas (Formato para YOLOv3Loss) ---")
    print(final_anchors)

    average_iou = calculate_average_iou(all_box_dimensions, anchor_centroids)
    print(f"\nIoU promedio de clustering: {average_iou:.4f}")

    print("\nGuarda estas anchor boxes para usarlas en la instanciación de tu modelo YOLOv3 y tu función de pérdida.")


Cargando anotaciones desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset\annotations.csv
Total de 4886 bounding boxes válidas cargadas para clustering.

Ejecutando K-means para encontrar 9 anchor boxes...
Iniciando K-means con 9 centroides iniciales:
[[ 98.  59.]
 [ 71.  77.]
 [116.  89.]
 [106.  83.]
 [128. 131.]
 [108. 101.]
 [104.  99.]
 [126.  73.]
 [ 34.  39.]]


Ejecutando K-means:  25%|██▍       | 74/300 [14:06<43:06, 11.44s/it]  

K-means convergió en la iteración 75.

--- Anchor Boxes Calculadas (Formato para YOLOv3Loss) ---
[[(227, 210), (179, 155), (124, 111)], [(105, 113), (104, 96), (80, 109)], [(112, 75), (87, 82), (39, 38)]]



IoU promedio de clustering: 0.8765

Guarda estas anchor boxes para usarlas en la instanciación de tu modelo YOLOv3 y tu función de pérdida.


In [2]:
# Otra version del mismo script

import numpy as np
import torch
from tqdm import tqdm # Para barras de progreso
import random # Para inicialización de K-means

# Asegúrate de que la función intersection_over_union esté accesible
# Si la tienes en utils.py, impórtala:
# from utils import intersection_over_union
# Si no, la definimos aquí para que el script sea autocontenido:
def intersection_over_union(boxes_preds, boxes_labels, box_format="midpoint"):
    if box_format == "midpoint":
        box1_x1 = boxes_preds[..., 0:1] - boxes_preds[..., 2:3] / 2
        box1_y1 = boxes_preds[..., 1:2] - boxes_preds[..., 3:4] / 2
        box1_x2 = boxes_preds[..., 0:1] + boxes_preds[..., 2:3] / 2
        box1_y2 = boxes_preds[..., 1:2] + boxes_preds[..., 3:4] / 2
        
        box2_x1 = boxes_labels[..., 0:1] - boxes_labels[..., 2:3] / 2
        box2_y1 = boxes_labels[..., 1:2] - boxes_labels[..., 3:4] / 2
        box2_x2 = boxes_labels[..., 0:1] + boxes_labels[..., 2:3] / 2
        box2_y2 = boxes_labels[..., 1:2] + boxes_labels[..., 3:4] / 2
    elif box_format == "corners":
        box1_x1 = boxes_preds[..., 0:1]
        box1_y1 = boxes_preds[..., 1:2]
        box1_x2 = boxes_preds[..., 2:3]
        box1_y2 = boxes_preds[..., 3:4]
        
        box2_x1 = boxes_labels[..., 0:1]
        box2_y1 = boxes_labels[..., 1:2]
        box2_x2 = boxes_labels[..., 2:3]
        box2_y2 = boxes_labels[..., 3:4]
    else:
        raise ValueError("box_format debe ser 'midpoint' o 'corners'")

    # Calcular las coordenadas del rectángulo de intersección
    # Aseguramos que estas variables estén definidas después de los ifs
    x1_inter = torch.max(box1_x1, box2_x1)
    y1_inter = torch.max(box1_y1, box2_y1)
    x2_inter = torch.min(box1_x2, box2_x2)
    y2_inter = torch.min(box1_y2, box2_y2)

    # Calcular el área de intersección
    intersection = (x2_inter - x1_inter).clamp(0) * \
                    (y2_inter - y1_inter).clamp(0)

    box1_area = abs((box1_x2 - box1_x1) * (box1_y2 - box1_y1))
    box2_area = abs((box2_x2 - box2_x1) * (box2_y2 - box2_y1))

    union = box1_area + box2_area - intersection + 1e-6
    iou = intersection / union
    return iou


def iou_distance(box, centroid):
    """
    Calcula la "distancia" como 1 - IoU, para usar en K-means.
    Las cajas se asumen centradas en (0,0) para comparar solo dimensiones.
    Args:
        box (np.array): [width, height] de una bounding box.
        centroid (np.array): [width, height] de un centroide (anchor).
    Returns:
        float: 1 - IoU entre la caja y el centroide.
    """
    box_tensor = torch.tensor([[0.0, 0.0, box[0], box[1]]], dtype=torch.float32)
    centroid_tensor = torch.tensor([[0.0, 0.0, centroid[0], centroid[1]]], dtype=torch.float32)
    
    # Usamos box_format="corners" porque las estamos tratando como (x1, y1, x2, y2)
    # donde x1,y1 son 0 y x2,y2 son width, height.
    iou = intersection_over_union(box_tensor, centroid_tensor, box_format="corners").item()
    return 1 - iou # Queremos minimizar la "distancia", que es 1 - IoU

def calculate_average_iou(boxes, centroids):
    """
    Calcula el IoU promedio entre las cajas y sus centroides asignados.
    """
    total_iou = 0.0
    num_boxes = 0
    for box in boxes:
        best_iou = -1
        for centroid in centroids:
            iou = 1 - iou_distance(box, centroid)
            if iou > best_iou:
                best_iou = iou
        total_iou += best_iou
        num_boxes += 1
    return total_iou / num_boxes if num_boxes > 0 else 0.0


def kmeans_iou(boxes, k, max_iters=100, tol=1e-6, random_seed=42):
    """
    Implementación de K-means que utiliza 1 - IoU como métrica de distancia.
    Args:
        boxes (np.array): Array de dimensiones de bounding boxes (N, 2), donde N es el número de cajas.
                        Cada fila es [width, height].
        k (int): Número de clusters (número de anchor boxes a generar).
        max_iters (int): Número máximo de iteraciones.
        tol (float): Tolerancia para la convergencia.
        random_seed (int): Semilla para la reproducibilidad.
    Returns:
        np.array: Centroides finales (anchor boxes).
    """
    # 1. Inicialización de los centroides (K-means++ como aproximación simplificada: elige k puntos aleatorios)
    # Opcional: Para una inicialización más robusta, podrías implementar K-means++ completo
    random.seed(random_seed)
    centroids = np.array(random.sample(list(boxes), k)).astype(np.float32) # Convertir a float32

    print(f"Iniciando K-means con {k} centroides iniciales:")
    print(centroids)

    for iteration in tqdm(range(max_iters), desc="Ejecutando K-means"):
        # 2. Asignación: Asigna cada caja al centroide más cercano (con menor 1-IoU)
        clusters = [[] for _ in range(k)]
        for i, box in enumerate(boxes):
            distances = [iou_distance(box, centroid) for centroid in centroids]
            closest_centroid_idx = np.argmin(distances)
            clusters[closest_centroid_idx].append(box)

        # 3. Actualización: Recalcula los centroides (promedio de las cajas asignadas)
        new_centroids = np.zeros_like(centroids)
        for i, cluster in enumerate(clusters):
            if cluster: # Evitar división por cero si un cluster está vacío
                new_centroids[i] = np.mean(cluster, axis=0)
            else:
                # Si un cluster está vacío, re-inicializarlo con una caja aleatoria
                new_centroids[i] = random.choice(list(boxes))

        # 4. Comprobar convergencia
        # Calculamos el desplazamiento máximo de los centroides
        # Usamos 1 - IoU como "distancia" entre centroides para la convergencia
        max_centroid_shift = 0.0
        for i in range(k):
            shift = iou_distance(centroids[i], new_centroids[i])
            if shift > max_centroid_shift:
                max_centroid_shift = shift
        
        centroids = new_centroids

        if max_centroid_shift < tol:
            print(f"K-means convergió en la iteración {iteration + 1}.")
            break
    else:
        print(f"K-means alcanzó el número máximo de iteraciones ({max_iters}) sin converger con la tolerancia {tol}.")

    return centroids

if __name__ == "__main__":
    # --- Configuración ---
    DATA_ROOT = 'C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset'
    CSV_FILE = os.path.join(DATA_ROOT, 'annotations.csv')
    NUM_ANCHORS = 9 # Para YOLOv3, suelen ser 9 anchors
    
    # --- 1. Cargar las dimensiones de todas las bounding boxes ---
    print(f"Cargando anotaciones desde: {CSV_FILE}")
    try:
        annotations_df = pd.read_csv(CSV_FILE)
        # Asegurarse de que no haya NaN y convertir a int
        annotations_df = annotations_df.dropna(subset=['xmin', 'ymin', 'xmax', 'ymax'])
        annotations_df[['xmin', 'ymin', 'xmax', 'ymax']] = annotations_df[['xmin', 'ymin', 'xmax', 'ymax']].astype(int)
    except FileNotFoundError:
        print(f"Error: Archivo CSV no encontrado en {CSV_FILE}")
        exit()

    # Calcular ancho y alto de cada caja
    # original_w y original_h son necesarios para desnormalizar si el CSV ya está normalizado.
    # En este dataset BCCD, las coordenadas son en píxeles.
    annotations_df['width'] = annotations_df['xmax'] - annotations_df['xmin']
    annotations_df['height'] = annotations_df['ymax'] - annotations_df['ymin']

    # Filtrar cajas degeneradas (ancho o alto <= 0)
    filtered_boxes = annotations_df[(annotations_df['width'] > 0) & (annotations_df['height'] > 0)]

    # Extraer solo las dimensiones [width, height]
    # Asegúrate de que las dimensiones sean coherentes con tu img_size si están normalizadas.
    # Si tu CSV tiene (x,y,w,h) normalizados, deberías multiplicarlos por el tamaño de la imagen original
    # para obtener dimensiones en píxeles para el clustering.
    # Como BCCD tiene píxeles, ya está bien.
    all_box_dimensions = filtered_boxes[['width', 'height']].values.astype(np.float32)

    if len(all_box_dimensions) == 0:
        print("No se encontraron bounding boxes válidas para el clustering. Revisa tu CSV y filtros.")
        exit()

    print(f"Total de {len(all_box_dimensions)} bounding boxes válidas cargadas para clustering.")

    # --- 2. Ejecutar K-means con métrica IoU ---
    print(f"\nEjecutando K-means para encontrar {NUM_ANCHORS} anchor boxes...")
    anchor_centroids = kmeans_iou(all_box_dimensions, k=NUM_ANCHORS, max_iters=300, tol=1e-5)

    # --- 3. Ordenar y Formatear los resultados ---
    # Ordenar los anchors por tamaño (ej. área o ancho) para asignarlos a las escalas.
    # YOLOv3 típicamente usa los 3 más grandes para la escala más pequeña (grid 13x13),
    # los 3 medianos para la escala media (grid 26x26), y los 3 más pequeños para la escala más grande (grid 52x52).
    
    # Ordenar por área para asegurar que los anchors más grandes vayan a la escala adecuada.
    # anchor_centroids es np.array([[w1,h1], [w2,h2], ...])
    anchor_centroids = anchor_centroids[np.argsort(anchor_centroids[:, 0] * anchor_centroids[:, 1])]
    
    # Dividir en 3 grupos (para 3 escalas) y formatear
    # Asignación sugerida:
    # Scale 0 (grid más pequeño, stride 32): 3 anchors más grandes
    # Scale 1 (grid medio, stride 16): 3 anchors medianos
    # Scale 2 (grid más grande, stride 8): 3 anchors más pequeños

    # Invertir el orden para que los más grandes estén primero
    anchor_centroids = anchor_centroids[::-1] 

    # Agrupar para las 3 escalas de YOLOv3 (si NUM_ANCHORS es 9)
    # anchors_per_scale = NUM_ANCHORS // 3
    # anchors_scale0 = anchor_centroids[0:anchors_per_scale]
    # anchors_scale1 = anchor_centroids[anchors_per_scale:2*anchors_per_scale]
    # anchors_scale2 = anchor_centroids[2*anchors_per_scale:3*anchors_per_scale]

    # Para YOLOv3, se suelen usar 3 anclas por escala. Las anclas se pasan a la Loss Function
    # como una lista de listas de tuplas: [[(w,h), (w,h), (w,h)], ... ]
    # y en el orden de las escalas de la red:
    # [anchors_for_scale_0 (largest grid), anchors_for_scale_1 (medium grid), anchors_for_scale_2 (smallest grid)]
    # NO: el orden de los anclas es Smallest grid -> largest objects. Esto significa que la primera sub-lista
    # de ANCHORS debería corresponder a la capa de salida con el stride más grande (e.g., 13x13 grid for 416x416).
    # Esa capa detecta objetos grandes.

    # Ordenarlos por área y agruparlos de mayor a menor para asignarlos a las escalas
    # Los más grandes (índices 0, 1, 2) van a la escala 0 (grid 13x13)
    # Los medianos (índices 3, 4, 5) van a la escala 1 (grid 26x26)
    # Los más pequeños (índices 6, 7, 8) van a la escala 2 (grid 52x52)

    # Aseguramos el orden de las anclas de mayor a menor
    sorted_anchors = []
    for anchor in anchor_centroids:
        sorted_anchors.append(tuple(anchor.round(2).astype(int))) # Redondeamos y convertimos a entero para tuplas

    # Formato final para la Loss Function
    # Este es el formato de ANCHORS que se pasa a YOLOv3Loss
    final_anchors = [
        sorted_anchors[0:3], # Las 3 más grandes (para grid 13x13)
        sorted_anchors[3:6], # Las 3 medianas (para grid 26x26)
        sorted_anchors[6:9], # Las 3 más pequeñas (para grid 52x52)
    ]
    
    print("\n--- Anchor Boxes Calculadas (Formato para YOLOv3Loss) ---")
    print(final_anchors)

    average_iou = calculate_average_iou(all_box_dimensions, anchor_centroids)
    print(f"\nIoU promedio de clustering: {average_iou:.4f}")

    print("\nGuarda estas anchor boxes para usarlas en la instanciación de tu modelo YOLOv3 y tu función de pérdida.")


Cargando anotaciones desde: C:/Users/gtoma/Master_AI_Aplicada/GitHubRep/PyTorch-YOLOv3/dataset\annotations.csv
Total de 4886 bounding boxes válidas cargadas para clustering.

Ejecutando K-means para encontrar 9 anchor boxes...
Iniciando K-means con 9 centroides iniciales:
[[ 98.  59.]
 [ 71.  77.]
 [116.  89.]
 [106.  83.]
 [128. 131.]
 [108. 101.]
 [104.  99.]
 [126.  73.]
 [ 34.  39.]]


Ejecutando K-means:  25%|██▍       | 74/300 [09:20<28:32,  7.58s/it]

K-means convergió en la iteración 75.

--- Anchor Boxes Calculadas (Formato para YOLOv3Loss) ---
[[(227, 210), (179, 155), (124, 111)], [(105, 113), (104, 96), (80, 109)], [(112, 75), (87, 82), (39, 38)]]



IoU promedio de clustering: 0.8765

Guarda estas anchor boxes para usarlas en la instanciación de tu modelo YOLOv3 y tu función de pérdida.
